In [ ]:
# !pip install cfgrib s3fs h5netcdf h5pyd

In [ ]:
# Setup
import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import io
import requests
import s3fs
import subprocess
from datetime import datetime

start_time = "2022-06-01 00:00"
end_time = "2022-06-30 00:00"

In [ ]:
# fmt = "%Y-%m-%d %H:%M"  # Date string format
    
# # Calculate time diff in hours
# days_diff = datetime.strptime(end_time, fmt)-datetime.strptime(start_time, "%Y-%m-%d %H:%M")
# times = (days_diff.days+1)*24
# print(times)

In [ ]:
# # Read in list of RAWS Stations
# df = pd.read_csv("raws_stations_WA.csv")

# # Filter to those with complete fmda data
# df = df[(df[['air_temp', 'relative_humidity', 'precip_accum',
#        'fuel_moisture', 'wind_speed', 'solar_radiation']]==1).sum(axis=1)==6]

# # Get list of coords
# points = list(df[["lon","lat"]].itertuples(index=False,name=None))
# names = np.unique(df['STID'])

# print(df.shape)

In [ ]:
from data_funcs import download_grib

In [ ]:
"https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140730/conus/hrrr.t18z.wrfsubhf0015.grib2"

In [ ]:
download_grib(
    source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.",
    time = start_time,
    model = "t18z.wrfsubhf",
    dest_dir =  "data" # destination subdirectory for url content
)

In [ ]:
# os.remove("data/hrrr.20140830.grib2")

In [ ]:
end_time = "2022-06-01 05:00"

time1 = datetime.strptime(start_time, fmt)
time2 = datetime.strptime(end_time, fmt)
dates = pd.date_range(start=time1,end=time2, freq="1H") # Series of dates
    
time = dates[1].strptime("%Y-%M-%D %H:%M")
    
download_grib(
    source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.",
    time = time,
    model = "t18z.wrfsubhf",
    dest_dir =  "data" # destination subdirectory for url content
)

In [ ]:
# gather_hrrr_time_range(
#     start=start_time,
#     end=end_time
# )